In [1]:
import numpy as np
import pymesh as pm

In [2]:
path = 'objs/'
name = 'apc_red_bowlx2'
# name = 'bunnyx2'
name = 'apc_1x2.5'
name = 'apc_redcupx2'
name = 'apc_2x2'
model = path + name  + '_decomp256.obj'
mesh = pm.meshio.load_mesh(model)

In [3]:
# box = pm.generate_box_mesh()

# save a stl file.
# pm.meshio.save_mesh('tmp.stl', mesh)


In [4]:
data_str = np.loadtxt(model, dtype=str)
data = np.array(data_str[:,1:], dtype=float)
bool_vf = np.char.find(data_str[:,0], 'v')
clip_num = [0]
rows = data.shape[0]
for i in range(rows-1):
    if data_str[i,0] != data_str[i+1,0]:
        clip_num.append(i+1)

clip_num.append(rows)

In [5]:
file_nums = int((len(clip_num) -1) /2)
import os
os.makedirs(path+ name + '/', exist_ok=True)

In [6]:
nums = len(clip_num)


for i in range(file_nums):
    v = data[clip_num[i*2] : clip_num[i*2+1],:]
    f = data[clip_num[i*2+1] : clip_num[i*2+2],:]
    f = np.array(f - np.min(f) +1, dtype=int)
    mesh_i = pm.form_mesh(v, f)
    # pm.meshio.save_mesh(path+ name +'_'+ str(i) + '.stl', mesh_i) # don't know why this doesn't work.
    vf_str = data_str[clip_num[i*2]:clip_num[i*2+2],0 ]
    vf = np.concatenate([v,f])
    np.savetxt(path+ name + '/' + name +'_'+ str(i) + '.obj', np.concatenate([vf_str.reshape(-1,1), vf], axis=1), fmt="%s")



In [7]:
for i in range(file_nums):
    mesh_i = pm.meshio.load_mesh(path+ name + '/' + name +'_'+ str(i) + '.obj')
    pm.meshio.save_mesh(path+ name + '/' + name +'_'+ str(i) + '.stl', mesh_i)

In [8]:
# delete all .obj files
files = os.listdir( path+ name + '/' )
for file in files:
    if file.endswith(".obj"):
        os.remove( os.path.join(  path+ name + '/', file ) )


In [9]:
# # start to generate .xml file for Mujoco.
# import xml.etree.cElementTree as ET
#
# root = ET.Element("name")
# doc = ET.SubElement(root, "asset")
# for i in range(file_nums):
#     ET.SubElement(doc, "mesh", name= name +'_'+ str(i), name = )
#
#
#


In [10]:
path + name

'objs/apc_2x2'

In [11]:
with open(path + name + '_preloads' + ".xml", "w") as f:
    start = '<'+ name + '>\n    <asset>\n'
    f.write(start)
    for i in range(file_nums):
        line = '        <mesh name=' + "\"" + name + '_'+ str(i)  + "\"" + ' file=' + "\"" +'objs/'+  name + '/' + name +'_'+ str(i) + '.stl'   + "\"" + ' /> \n'
        # print(line)
        f.write(line)
    end_str = '    </asset>\n</'+ name + '>'
    f.write(end_str)

In [13]:
if name[-2]=='x':
    # name_str = name[:-2]
    name_str = name
    # pass
else:
    name_str = name

geom = "\" class=\"obj_pieces\" pos=\"0 0 0\" quat=\"1 0 0 0\" type=\"mesh\" mesh="
with open(path + name + '_bodies' ".xml", "w") as f:
    start = '<'+ name + '>\n'
    f.write(start)
    for i in range(file_nums):
        # <geom pos="0.0425 -0.0425 0" quat="-0.707107 0 0.707107 0" type="mesh"  group="1" rgba="0.2 0.2 0.2 1" mesh="allegro_mount" />
        line = "    <geom name=\"" + name_str + '_'+ str(i)    + geom + '\"' +  name + '_'+ str(i)   + '\"/>\n'
        f.write(line)
    end_str = '</'+ name + '>'
    f.write(end_str)